In [ ]:
# standard imports
import numpy as np
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
# import geometry classes
from geometry.node import node
from geometry.cell import cell
from geometry.mesh import rot_sym_mesh
from problem.isentropic_navier_stokes import convection, diffusion, skalar_product, curl

# utility functions
from utilities.utilities import *
from utilities.pod import get_POD, get_activations

# Setting Custom Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./mesh_files/mesh_cylinder_quad.su2",file_format="su2")
# convert mesh into custom class exp_mesh
mesh = rot_sym_mesh(meshio_mesh)

# Geometric Constants

In [ ]:
# DEFINING CONSTANTS
n = mesh.n                  # number of nodes
N = mesh.N                  # number of cells
T = 220                     # number of snapshots
alpha = 1                   # inner product coefficient  # method for derivative computation
gamma = 1.4                 # isentropic coefficient -> material property
max_POD = 40                # maximum number of POD modes
mach = 0.6                  # mach number
sos = 340.29                # isentropic speed of sound
u_free = mach * sos         # freestream velocity
Re = 100                    # Reynolds number of freeflow
dt = 0.0001                 # simulation time step
include_avg = False         # include average flow in POD computation
nondim = False              # apply nondimensionalization to results
subsampling = 6             # subsampling for data
stabilization = "viscosity" # stabilization method for galerkin system
controlled = False          # activate controlled system
tau = 0                     # penalty term coefficient

# Data Preparation

In [ ]:
# reading file data
data_dir  = "./flow_data/lam_cyl_comp_air/mach 0.6/"
data_dict = read_csv_data(data_dir,delay=0,subsampling=subsampling,max_size=T)

# geometric information
x = data_dict["x"][:,0] - 0.5
y = data_dict["y"][:,0]

# fluid velocity and mach number
u = np.divide(data_dict['Momentum_x'], data_dict['Density'])
v = np.divide(data_dict['Momentum_y'], data_dict['Density'])
a = data_dict['Mach']

# construct state based data
q = np.vstack([u,v,a])
T = q.shape[1]
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),T,axis=1)

# data for pod decomposition
if include_avg:
    pod_data = q
else:
    pod_data = q_fluc

### set scalar weights for multiplicative computation
# volume participations
volume_weight = mesh.volume_weights
# additional weight for speed of sound
mach_weight = (2 * alpha / (gamma - 1))
# construction of weights
inner_weight = np.concatenate([np.ones(n),np.ones(n),mach_weight*np.ones(n)]) * volume_weight

# POD Decomposition

In [ ]:
[pod_modes, S] = get_POD(pod_data,inner_weight,20)
coeffs = get_activations(pod_data,pod_modes,inner_weight,20)
if nondim:
    coeffs /= u_free

In [ ]:
# energy reconstruction threshold
epsilon = 0.99
acc = 0
num = 0
while acc < epsilon:
    num+=1
    acc = sum(S[:num])/sum(S)    
print("Number of eigenvectors necessary for thresholded reconstruction: \t" + str(num))
if num % 2 == 1:
    num+=1

# manual value
num = 8

In [ ]:
# plot_activations(coeffs,8,dt=dt*subsampling)

# Control Data Input

In [ ]:
control_dir  = "./flow_data/lam_cyl_comp_air/potVortex_0.6_1/"
control_dict = read_csv_data(control_dir,0,1,1)

u_con = np.divide(control_dict['Momentum_x'], control_dict['Density'])
v_con = np.divide(control_dict['Momentum_y'], control_dict['Density'])
a_con = control_dict['Mach']

# construct state based data
q_con = np.vstack([u_con,v_con,a_con]).squeeze()

# Control Function Method
Additional coefficients in Galerkin system

In [ ]:
# Computation of Galerkin control terms
Lcon = diffusion(mesh,q_con)
Qcon = convection(mesh,q_con,q_con)
Qconavg_1 = convection(mesh,q_con,q_avg)
Qconavg_2 = convection(mesh,q_avg,q_con)

# initialize arrays
d1 = np.empty(num)
d2 = np.empty(num)
f = np.empty(num)
g = np.empty((num,num))
h = np.empty(num)

# compute temporary L and Q operators for projection
tmp_1 = np.empty((num,3*n))
tmp_2 = np.empty((num,3*n))
for i in tqdm(range(num)):
    tmp_1[i] = convection(mesh,q_con,pod_modes[:,i])
    tmp_2[i] = convection(mesh,pod_modes[:,i],q_con)

# compute finale coefficients
for k in range(num):
    d1[k] = skalar_product(Lcon,pod_modes[:,k],inner_weight)
    d2[k] = skalar_product(Qconavg_1 + Qconavg_2,pod_modes[:,k],inner_weight)
    f[k] = skalar_product(Qcon,pod_modes[:,k],inner_weight)
    h[k] = skalar_product(q_con,pod_modes[:,k],inner_weight)
    for i in range(num):
        g[k,i] = skalar_product(tmp_1[i] + tmp_2[i],pod_modes[:,k],inner_weight)

# Galerkin Coefficients

In [ ]:
# Galerkin system coefficients
# Computation of Galerkin system parameters
Qavg = convection(mesh,q_avg,q_avg)
Lavg = diffusion(mesh,q_avg)

# initialize arrays for parameters
b1 = np.empty(num)
b2 = np.empty(num)
L1 = np.empty((num,num))
L2 = np.empty((num,num))
Q_ = [np.empty((num,num,)) for x in range(num)]

# compute L and Q operators for projection
Q_tmp1 = np.empty((num,3*n))
Q_tmp2 = np.empty((num,3*n))
Q_tmp3 = np.empty((num,num,3*n))
L_tmp = np.empty((num,3*n))

for i in tqdm(range(num)):
    Q_tmp1[i] = convection(mesh,q_avg,pod_modes[:,i])
    Q_tmp2[i] = convection(mesh,pod_modes[:,i],q_avg)
    L_tmp[i] = diffusion(mesh,pod_modes[:,i])
    for j in range(num):
        Q_tmp3[i,j] = convection(mesh,pod_modes[:,i],pod_modes[:,j])

# compute ODE coefficients
for k in tqdm(range(num)):
    b1[k] = skalar_product(Lavg,pod_modes[:,k],inner_weight)
    b2[k] = skalar_product(Qavg,pod_modes[:,k],inner_weight)
    for i in range(num):
        L1[k,i] = skalar_product(L_tmp[i],pod_modes[:,k],inner_weight)
        L2[k,i] = skalar_product(np.add(Q_tmp1[i],Q_tmp2[i]),pod_modes[:,k],inner_weight)
        for j in range(num):
            Q_[k][i,j] = skalar_product(Q_tmp3[i,j],pod_modes[:,k],inner_weight)

# Galerkin System

In [ ]:
# initial conditions
a0 = coeffs[:num,0]

# temporal problem setup
if mach == 0.6:
    dt_ref = 0.0001 * subsampling
    dt = 0.001
    shed_period = 0.03

elif mach == 0.1:
    dt_ref = 0.001 * subsampling
    dt = 0.001
    shed_period = 0.18

elif mach == 0.01:
    dt_ref = 0.02 * subsampling
    dt = 0.02
    shed_period = 1.84

temporal_factor = 1

# temporal span for solver
t0 = 0
tmax = shed_period * 20 # 100

sampling_span = np.linspace(t0,tmax-dt,num=int(tmax/dt))

# dynamic viscosity for stabilization
if stabilization=="viscosity":
    if mach == 0.6:
        # tuned
        if num == 10:
            nu = 4.09   # 10 modes
        elif num == 8:
            nu = 3.24   # 8 modes
        elif num == 6:
            nu = 3.07   # 6 modes
        elif num == 4:
            nu = 3.625  # 4 modes
        elif num == 2:  
            nu = 4.36   # 2 modes

    elif mach == 0.1:
        # tuned
        if num == 10:
            nu = 0.745   # 10 modes
        elif num == 8:
            nu = 0.7445  # 8 modes
        elif num == 6:
            nu = 0.7390  # 6 modes
        elif num == 4:
            nu = 0.7635  # 4 modes
        elif num == 2:  
            nu = 0.8625  # 2 modes

    elif mach == 0.01:
        # tuned
        if num == 10:
            nu = 0.1535  # 10 modes
        elif num == 8:
            nu = 0.151   # 8 modes
        elif num == 6:
            nu = 0.1515  # 6 modes
        elif num == 4:
            nu = 0.1515  # 4 modes
        elif num == 2:  
            nu = 0.1550  # 2 modes

In [ ]:
# defining ode function for node based coefficients
def galerkin_system(t,a):
    # set global variables references
    global Q_, L1, L2, b1, b2, nu
    a_dot = np.empty_like(a)
    for k in range(a_dot.shape[0]):
        a_dot[k] = (nu * b1[k] + b2[k] + np.inner((nu * L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_[k]),np.expand_dims(a,1))) 
    return a_dot

In [ ]:
sol = solve_ivp(galerkin_system,(t0,tmax),a0,method='LSODA',t_eval=sampling_span)
sol.message

# Comparison of Limit Cycles

In [ ]:
# Galerkin model 
d = int(num/2)
fig,ax = plt.subplots(d,2,figsize=(15,10))
fig.tight_layout(pad=3.0)

t = sampling_span 

# add plots over full time domain
for i in range(num):
    if num <= 2:
        ax[int(i%2)].plot(t[-100:],sol.y[i,-100:len(t)]/(sos * mach))
        ax[int(i%2)].title.set_text("Activation for Eigenflow " + str(i))
    else:
        ax[int(i/2)][int(i%2)].plot(t[-100:],sol.y[i,-100:len(t)]/(sos * mach))
        ax[int(i/2)][int(i%2)].title.set_text("Activation for Eigenflow " + str(i))
#plt.savefig("mach_0.6_stabilized_av.png")

# Reconstruction

In [ ]:
# reconstruct flows
rec_len = 100
ssubsampling = int(shed_period / dt_ref / 20)
rec = np.repeat(np.expand_dims(q_avg,1),rec_len,1)
for i in range(num):
    rec += np.outer(pod_modes[:,i],sol.y[i,-100*ssubsampling::ssubsampling])

In [ ]:
for i in range(100):
    data = rec[0*n:1*n,i]/(sos * mach)
    data[0] = q[:n,0].max()/(sos * mach) * 1.8
    data[1] = q[:n,0].min()/(sos * mach)
    plot_cylinder_data(x,y,data,zoom=True,cmap='twilight')
    #fac = int(0.03 / (0.0001 * subsampling) / 20)
    #plot_cylinder_data(x,y,q[0*n:1*n,i*fac]/(sos * mach), zoom=True, cmap='twilight')
    #plt.savefig(str(i)+".png")
    plt.close('all')

# Optimal Control - Controlled System Dynamics

In [ ]:
def controlled_galerkin_system(t,a):
    # set global variables references
    global Q_, L1, L2, b1, b2, nu
    # 
    a_dot = np.empty_like(a)
    for k in range(a_dot.shape[0]):
        a_dot[k] = (nu * b1[k] + b2[k] + np.inner((nu*L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_[k]),np.expand_dims(a,1))) + gamma(t) * (nu * d1[k] + d2[k] + np.inner(g[k],a) + gamma(t) * f[k]) + h[k] * dgamma(t)
    return a_dot

# parameterized control function - Active Control paper
design_dim = 100
design_vars = np.ones(design_dim)
gamma_start = 0

# temporal span for solver
t0 = 0
tmax = shed_period * 20 
Nt = int(tmax/dt)
sampling_span = np.linspace(t0,tmax-dt,num=int(tmax/dt))


def gamma(t):
    global shed_period, design_dim, gamma_start
    T = shed_period
    tmp = gamma_start * (1 - t / T) 
    for i in range(design_dim):
        tmp += design_vars[i] * np.sin(i * np.pi * t / (T * 2))
    return tmp

def dgamma(t):
    global shed_period, design_dim, gamma_start
    tmp = - gamma_start * 1 / T 
    for i in range(design_dim):
        tmp += design_vars[i] * np.cos(i * np.pi * t / (T * 2)) * (i * np.pi / (T * 2))
    return tmp

def discrete_gradient(a):
    global dt, tmax, Nt, A, h, d1, d2, g, f, phi, gamma, dfdgamma, dfda ,design_dim, num
    grad_tmp = np.zeros(design_dim)
    for m in range(design_dim): # design variables
        for n in range(Nt): # timesteps
            tmp = 0
            for k in range(num):
                tmp_ = 0
                for i in range(num):
                    tmp_ += h[k] * A[n][k,i] * phi[i] + g[k,i] * a[n][i]
                tmp += (tmp_ + nu * d1[k] + d2[k] + 2 * gamma(t)) * phi[n][k] - h[k] * dfda[n][k]
            tmp += dfdgamma[n]
            grad_tmp[m] += tmp * sin(m * n * np.pi * dt / (2 * tmax))
    return dt * grad_tmp

AD Zyklus:
 - Gamma paramterisieren -> $\beta_m$
 - $\beta_m$ als Designvariablen deklarieren
 
Zyklus:
   - $\gamma(\beta_m)$ bestimmen
   - $a(\gamma)$ bestimmen
   - Kostenfunktion $(a,\gamma)$ berechnen
   - AD mit codipack
   - $\beta$ mit AD Gradient updaten

Optimierung Peraire:
- $\gamma(\beta_m)$ 
- $a(\gamma)$ bestimmen
- $\phi_k$ über zusätzliche ODE lösen
- Gradientenfunktion( $a_k,\gamma,\phi_k$ ) bestimmen
- Update $\beta_k$

In [ ]:
sol = solve_ivp(controlled_galerkin_system,(t0,tmax),a0,method='LSODA',t_eval=sampling_span)
sol.message

In [ ]:
# Galerkin model 
d = int(num/2)
fig,ax = plt.subplots(d,2,figsize=(15,10))
fig.tight_layout(pad=3.0)

t = sampling_span 

# add plots over full time domain
for i in range(num):
    if num <= 2:
        ax[int(i%2)].plot(t[-100:],sol.y[i,-100:len(t)]/(sos * mach))
        ax[int(i%2)].title.set_text("Activation for Eigenflow " + str(i))
    else:
        ax[int(i/2)][int(i%2)].plot(t[-100:],sol.y[i,-100:len(t)]/(sos * mach))
        ax[int(i/2)][int(i%2)].title.set_text("Activation for Eigenflow " + str(i))

In [ ]:
# reconstruct flows
rec_len = 100
ssubsampling = int(shed_period / dt_ref / 20)
rec = np.repeat(np.expand_dims(q_avg,1),rec_len,1)
for i in range(num):
    rec += np.outer(pod_modes[:,i],sol.y[i,-100*ssubsampling::ssubsampling])

for i in range(rec_len):
    t = sampling_span[-100*ssubsampling::ssubsampling][i]
    rec[:,i] += gamma(t) * q_con

In [ ]:
plot_cylinder_data(x,y,rec[:n,22],zoom=True,cmap='twilight')